<font color = green >

## Home Task 

</font>


The Task is to extract relevant infromation from the messy medical data using regex.

Each line of the `dates.txt` file corresponds to a medical note. Each note has a date that needs to be extracted, but each date is encoded in one of many formats.

The goal is to correctly identify all of the different date variants encoded in this dataset and to properly normalize and sort the dates. 

1) Extract the date strings
Here is a list of some of the variants you might encounter in this dataset:
* 04/20/2009; 04/20/09; 4/20/09; 4/3/09
* Mar-20-2009; Mar 20, 2009; March 20, 2009;  Mar. 20, 2009; Mar 20 2009;
* 20 Mar 2009; 20 March 2009; 20 Mar. 2009; 20 March, 2009
* Mar 20th, 2009; Mar 21st, 2009; Mar 22nd, 2009
* Feb 2009; Sep 2009; Oct 2010
* 6/2008; 12/2009
* 2009; 2010

2) Normalize the extracted dates considering the following rules:  
* Assume all dates in xx/xx/xx format are mm/dd/yy
* Assume all dates where year is encoded in only two digits are years from the 1900's (e.g. 1/5/89 is January 5th, 1989)
* If the day is missing (e.g. 9/2009), assume it is the first day of the month (e.g. September 1, 2009).
* If the month is missing (e.g. 2010), assume it is the first of January of that year (e.g. January 1, 2010).

3) Sort records in ascending chronological order accoring.





In [1]:
import pandas as pd

with open('dates.txt') as file:
    doc = [note for note in file]
df = pd.Series(doc)

df.head(10)

0         03/25/93 Total time of visit (in minutes):\n
1                       6/18/85 Primary Care Doctor:\n
2    sshe plans to move as of 7/8/71 In-Home Servic...
3                7 on 9/27/75 Audit C Score Current:\n
4    2/6/96 sleep studyPain Treatment Pain Level (N...
5                    .Per 7/06/79 Movement D/O note:\n
6    4, 5/18/78 Patient's thoughts about current su...
7    10/24/89 CPT Code: 90801 - Psychiatric Diagnos...
8                         3/7/86 SOS-10 Total Score:\n
9             (4/10/71)Score-1Audit C Score Current:\n
dtype: object

In [2]:
date_cols = ['Year', 'Month', 'Day']

mon_dic = {
    'jan': '01',
    'feb': '02',
    'mar': '03',
    'apr': '04',
    'may': '05',
    'jun': '06',
    'jul': '07',
    'aug': '08',
    'sep': '09',
    'oct': '10',
    'nov': '11',
    'dec': '12'
}


days = '|'.join([str(day) for day in range(1, 32)] + ['0' + str(day) for day in range(1, 10)])
mons = '|'.join(list(mon_dic.values()) + [str(month) for month in range(1, 10)])
regex_day = r'(?P<Day>' + days + r')'
regex_month_1 = r'(?P<Month>' + '|'.join(list(mon_dic.keys())) + r')'
regex_month_2 = r'(?P<Month>' + mons + r')'
regex_year_1 = r'(?P<Year>(?<![0-9])(?:(?:[789]{1}[0-9]{1})|(?:(?:19|20){1}[0-9]{2}))(?![0-9]))'
regex_year_2 = r'(?P<Year>(?<![0-9])(?:(?:19|20){1}[0-9]{2})(?![0-9]))'
regexs = [
    r'(?P<Date>' + regex_day + r' +' + regex_month_1 + r'[a-z.,]* +' + regex_year_1 + r')',
    r'(?P<Date>' + regex_month_1 + r'[a-z.,]*[ -]+' + regex_day + r'[a-z,]*[ -]+' + regex_year_1 + r')',
    r'(?P<Date>' + regex_month_2 + r'[-/]+' + regex_day + r'[-/]+' + regex_year_1 + r')',
    r'(?P<Date>' + regex_month_1 + r'[a-z.,]*[ -]+(?:' + regex_day + r'[a-z,]*[ -]+)?' + regex_year_1 + r')',
    r'(?P<Date>' + regex_month_2 + r'[-/]+(?:' + regex_day + r'[-/]+)?' + regex_year_1 + r')',
    r'(?P<Date>(?:' + regex_month_2 + r'[-/]+(?:' + regex_day + r'[-/]+)?)?' + regex_year_2 + r')'
]


In [3]:
def extract_fix_dates(notes, regex):

    dates = notes.str.lower().str.extract(regex)
    dates[date_cols[1:]] = dates[date_cols[1:]].fillna("1")
    dates = dates.dropna()

    dates['Month'] = dates['Month'].apply(
        lambda m: mon_dic[m] if m in mon_dic else m)

    dates[date_cols] = dates[date_cols].astype(int)

    dates['Year'] = dates['Year'].apply(lambda y: y + 1900 if y < 100 else y)

    return dates


def extract_fix_sort_dates(notes, regexs):

    remain_notes = notes.copy()
    dates_dfs = []

    for regex in regexs:
        tmp_df = extract_fix_dates(remain_notes, regex)
        dates_dfs.append(tmp_df)
        remain_notes = remain_notes.drop(tmp_df.index.values)

    sorted_dates = pd.concat(dates_dfs, sort=False).sort_values(date_cols)

    return sorted_dates, remain_notes

In [4]:
sorted_dates, remain_notes = extract_fix_sort_dates(df, regexs)
sorted_dates

,Date,Day,Month,Year
9,4/10/71,10,4,1971
84,5/18/71,18,5,1971
2,7/8/71,8,7,1971
53,7/11/71,11,7,1971
28,9/12/71,12,9,1971
...,...,...,...,...
427,5/2016,1,5,2016
141,30 may 2016,30,5,2016
186,13 oct 2016,13,10,2016
161,19 oct 2016,19,10,2016


In [5]:
sorted_dates['Year'] = sorted_dates['Year'].astype(int)
sorted_dates['Month'] = sorted_dates['Month'].astype(int)
sorted_dates['Day'] = sorted_dates['Day'].astype(int)

sorted_dates['res'] = pd.to_datetime(
    sorted_dates[['Year', 'Month', 'Day']].astype(str).agg('-'.join, axis=1))
result = sorted_dates[['res']].sort_values(by='res')
result['index'] = result.index
result = result.reset_index(drop=True)

result

,res,index
0,1971-04-10,9
1,1971-05-18,84
2,1971-07-08,2
3,1971-07-11,53
4,1971-09-12,28
...,...,...
495,2016-05-01,427
496,2016-05-30,141
497,2016-10-13,186
498,2016-10-19,161


<font color = blue >


### Expected Output

</font>

<table align= 'left'>
    <tr><td></td><td> res</td><td>index</td></tr>
    <tr><td>0</td><td> 1971-04-10</td><td>9</td></tr>    
    <tr><td>1</td><td>1971-05-18</td><td>84</td></tr>    
    <tr><td>2</td><td>1971-07-08</td><td>2</td></tr>    
    <tr><td>3</td><td>1971-07-11</td><td>53</td></tr>    
    <tr><td>4</td><td>1971-09-12</td><td>28</td></tr>    
    <tr><td>.</td></tr>    
    <tr><td>.</td></tr>    
    <tr><td>.</td></tr>    
    <tr><td>495 </td><td>2016-05-01</td><td>    427</td></tr>    
    <tr><td>496</td><td> 2016-05-30  </td><td>  141</td></tr>    
    <tr><td>497</td><td> 2016-10-13  </td><td>  186</td></tr>    
    <tr><td>498</td><td> 2016-10-19   </td><td> 161</td></tr>    
    <tr><td>499 </td><td>2016-11-01 </td><td>   413</td></tr>    
</table> 